<a href="https://www.kaggle.com/code/noahhaag/insurance-data-xgboost-w-tuning?scriptVersionId=212202353" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import cupy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv').drop(columns='id')
test=pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv').drop(columns='id')
test_id=pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')['id']

In [3]:
train.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
for i in train.columns:
    if train[i].dtypes == 'object':  
        train[i].fillna(train[i].mode()[0], inplace=True) 
    elif train[i].dtypes=='int64':
    	train[i].fillna(train[i].mean(),inplace=True)
    elif  train[i].dtypes=='float64':
    	train[i].fillna(train[i].mean(),inplace=True)

/tmp/ipykernel_23/1996321317.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[i].fillna(train[i].mean(),inplace=True)
/tmp/ipykernel_23/1996321317.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [5]:
for i in test.columns:
    if test[i].dtypes == 'object':  
        test[i].fillna(test[i].mode()[0], inplace=True) 
    elif test[i].dtypes=='int64':
    	test[i].fillna(test[i].mean(),inplace=True)
    elif  test[i].dtypes=='float64':
    	test[i].fillna(test[i].mean(),inplace=True)

/tmp/ipykernel_23/2849957383.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test[i].fillna(test[i].mean(),inplace=True)
/tmp/ipykernel_23/2849957383.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

In [6]:
train.isna().sum()

Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Insurance Duration      0
Policy Start Date       0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder
for j in train.columns:
    if train[j].dtypes=='object':
        train[j]=LabelEncoder().fit_transform(train[j])

for j in test.columns:
    if test[j].dtypes=='object':
        test[j]=LabelEncoder().fit_transform(test[j])

In [8]:
train.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,0,10049.0,1,1.0,0,1,22.598761,2,2,2.0,17.0,372.00000,5.0,145522,2,0,3,2,2869.0
1,39.0,0,31678.0,0,3.0,2,0,15.569731,0,1,1.0,12.0,694.00000,2.0,127832,0,1,1,2,1483.0
2,23.0,1,25602.0,0,3.0,1,1,47.177549,1,2,1.0,14.0,592.92435,3.0,137880,1,1,3,2,567.0
3,21.0,1,141855.0,1,2.0,0,0,10.938144,0,0,1.0,0.0,367.00000,1.0,161517,2,1,0,0,765.0
4,21.0,1,39651.0,2,1.0,0,1,20.376094,0,2,0.0,8.0,598.00000,4.0,76621,2,1,3,2,2022.0


In [9]:
from sklearn.preprocessing import minmax_scale

train['Income per year of age'] = train['Annual Income']/ train['Age']
test['Income per year of age'] = test['Annual Income']/ test['Age']

train['scaled health score'] = minmax_scale(train['Health Score'])
test['scaled health score'] = minmax_scale(test['Health Score'])

train['family demographics'] = train['Marital Status'] + train['Number of Dependents']
test['family demographics'] = test['Marital Status'] + test['Number of Dependents']

In [10]:
train.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Income per year of age,scaled health score,family demographics
0,19.0,0,10049.0,1,1.0,0,1,22.598761,2,2,...,5.0,145522,2,0,3,2,2869.0,528.894737,0.361397,2.0
1,39.0,0,31678.0,0,3.0,2,0,15.569731,0,1,...,2.0,127832,0,1,1,2,1483.0,812.256410,0.238002,3.0
2,23.0,1,25602.0,0,3.0,1,1,47.177549,1,2,...,3.0,137880,1,1,3,2,567.0,1113.130435,0.792879,3.0
3,21.0,1,141855.0,1,2.0,0,0,10.938144,0,0,...,1.0,161517,2,1,0,0,765.0,6755.000000,0.156695,3.0
4,21.0,1,39651.0,2,1.0,0,1,20.376094,0,2,...,4.0,76621,2,1,3,2,2022.0,1888.142857,0.322378,3.0


In [11]:
x=train.iloc[:,:-1]
y=train['Premium Amount']
x

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Income per year of age,scaled health score
0,19.0,0,10049.000000,1,1.000000,0,1,22.598761,2,2,...,372.00000,5.0,145522,2,0,3,2,2869.0,528.894737,0.361397
1,39.0,0,31678.000000,0,3.000000,2,0,15.569731,0,1,...,694.00000,2.0,127832,0,1,1,2,1483.0,812.256410,0.238002
2,23.0,1,25602.000000,0,3.000000,1,1,47.177549,1,2,...,592.92435,3.0,137880,1,1,3,2,567.0,1113.130435,0.792879
3,21.0,1,141855.000000,1,2.000000,0,0,10.938144,0,0,...,367.00000,1.0,161517,2,1,0,0,765.0,6755.000000,0.156695
4,21.0,1,39651.000000,2,1.000000,0,1,20.376094,0,2,...,598.00000,4.0,76621,2,1,3,2,2022.0,1888.142857,0.322378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,0,27316.000000,1,0.000000,2,2,13.772907,2,2,...,372.00000,3.0,124181,2,0,0,0,1303.0,758.777778,0.206459
1199996,54.0,1,35786.000000,0,2.009934,2,1,11.483482,0,1,...,597.00000,4.0,102449,2,0,3,0,821.0,662.703704,0.166268
1199997,19.0,1,51884.000000,0,0.000000,2,0,14.724469,1,0,...,592.92435,6.0,59151,1,0,1,1,371.0,2730.736842,0.223164
1199998,55.0,1,32745.217777,2,1.000000,3,0,18.547381,1,2,...,407.00000,4.0,69979,2,0,0,0,596.0,595.367596,0.290275


In [12]:
# Find minimum values
min_values = train.min()

# Find maximum values
max_values = train.max()

print("Minimum values:")
print(min_values)

print("\nMaximum values:")
print(max_values)

Minimum values:
Age                        18.000000
Gender                      0.000000
Annual Income               1.000000
Marital Status              0.000000
Number of Dependents        0.000000
Education Level             0.000000
Occupation                  0.000000
Health Score                2.012237
Location                    0.000000
Policy Type                 0.000000
Previous Claims             0.000000
Vehicle Age                 0.000000
Credit Score              300.000000
Insurance Duration          1.000000
Policy Start Date           0.000000
Customer Feedback           0.000000
Smoking Status              0.000000
Exercise Frequency          0.000000
Property Type               0.000000
Premium Amount             20.000000
Income per year of age      0.032787
scaled health score         0.000000
family demographics         0.000000
dtype: float64

Maximum values:
Age                           64.000000
Gender                         1.000000
Annual Income        

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2)

In [14]:
ytest.isna().sum()

0

# XGBoost

In [15]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

grid = {
    'n_estimators' : [100,125,50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1],
    'grow_policy' : ['depthwise', 'lossguide'],
    'booster' : ['gbtree', 'gblinear', 'dart']
}


# Create the model
xgb_reg = xgb.XGBRegressor(verbosity= 2, device = 'cuda')

grid_search = GridSearchCV(xgb_reg,
                           grid,
                           n_jobs=-1,
                           cv=3,
                           scoring='neg_mean_squared_log_error',
                          verbose=2)

# Train the model
grid_search.fit(xtrain, ytrain)
predXGB=grid_search.predict(xtest)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:39:32] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:39:32] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.5; total time=   6.1s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1; total time=   4.9s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=0.5; total time=   5.3s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=1; total time=   4.9s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.5; total time=   4.0s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.7; total time=   4.1s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.5; total time=   5.2s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=5, n_

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.5; total time=   5.7s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.7; total time=   4.9s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1; total time=   4.6s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=0.7; total time=   4.1s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=125, subsample=1; total time=   4.0s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.5; total time=   3.4s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=1; total time=   3.6s
[CV] END booster=gbtree, grow_policy=depthwise, learning_rate=0.1, max_depth=5, n_es

...

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:05:19] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[01:04:52] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:52] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:52] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:52] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:53] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:53] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:53] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:53] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:54] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:54] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:54] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:55] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:55] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:04:55] I

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:15:26] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[01:14:32] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:33] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:33] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:33] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:34] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:34] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:34] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:35] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:35] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:35] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:36] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:36] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:36] INFO: /workspace/src/gbm/gbtree.cc:887: drop 0 trees, weight = 1
[01:14:37] I

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [-2.40119389e-04 -2.26537916e-04 -2.27199194e-04 -2.15825689e-04
 -2.01444449e-04 -1.93780508e-04 -4.97048439e-03 -4.95800517e-03
 -4.89354142e-03 -1.10952565e-04 -1.10062258e-04 -1.10025817e-04
 -1.10633400e-04 -1.09827805e-04 -1.10030987e-04 -1.95775699e-03
 -1.94751267e-03 -1.94938992e-03 -1.09001499e-04 -1.08713833e-04
 -1.08521003e-04 -1.09131305e-04 -1.08848961e-04 -1.08685942e-04
 -1.75132617e-03 -1.74498505e-03 -1.73934301e-03 -5.81627372e-01
 -5.81376320e-01 -5.81519065e-01 -4.71383013e-01 -4.71267463e-01
 -4.71107417e-01 -8.79822520e-01 -8.79849890e-01 -8.79966631e-01
 -5.52178715e-01 -5.52152918e-01 -5.52143101e-01 -4.38321039e-01
 -4.38314630e-01 -4.38303512e-01 -8.62172227e-01 -8.62156990e-01
 -8.62145331e-01 -5.50909678e-01 -5.50823251e-01 -5.50762247e-01
 -4.36862231e-01 -4.36760139e-01 -4.36693819e-01 -8.61432398e-01
 -8.61391050e-0

In [16]:
import numpy as np

def rmsle(y_true, y_pred):
    # Ensure no negative values by adding 1 to predictions and true values
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate RMSLE
    log_true = np.log1p(y_true)
    log_pred = np.log1p(y_pred)
    squared_log_error = np.square(log_true - log_pred)
    rmsle_value = np.sqrt(np.mean(squared_log_error))
    
    return rmsle_value


rmsle_value = rmsle(ytest, predXGB)
print(f"RMSLE: {rmsle_value:.4f}")

RMSLE: 0.0105


# Submit 

## predict=predXGB

df=pd.DataFrame()
df['id']=test_id
df['Premium Amount']=predict

df.to_csv('ins_sub.csv',index=False)